Header
track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms


In [83]:
CREATE_SCHEMA = "CREATE SCHEMA Songs;"

In [84]:
CREATE_TABLE_ARTIST = "CREATE TABLE Songs.Artist ( \
    artist_name varchar(100) not null, \
    PRIMARY KEY (artist_name) \
);"

CREATE_TABLE_ALBUM = "CREATE TABLE Songs.Album ( \
    album_id varchar(30) not null, \
    album_name varchar(100) not null, \
    album_release_date date, \
    PRIMARY KEY (album_id) \
);"

CREATE_TABLE_GENRE = "CREATE TABLE Songs.Genre ( \
    genre_name varchar(30) not null, \
    PRIMARY KEY (genre_name) \
);"

CREATE_TABLE_PLAYLIST = "CREATE TABLE Songs.Playlist ( \
    playlist_id varchar(30) not null, \
    playlist_name varchar(100) not null, \
    playlist_genre varchar(30) not null, \
    playlist_subgenre varchar(30) not null, \
    PRIMARY KEY (playlist_id), \
    CONSTRAINT FK_Track_Genre FOREIGN KEY (playlist_genre) REFERENCES Songs.Genre (genre_name), \
    CONSTRAINT FK_Track_SubGenre FOREIGN KEY (playlist_subgenre) REFERENCES Songs.Genre (genre_name) \
);"

CREATE_TABLE_TRACK = "CREATE TABLE Songs.Track ( \
    track_id varchar(30) not null, \
    track_name varchar(100) not null, \
    track_artist varchar(100) not null, \
    track_popularity decimal(2), \
    track_album_id varchar(30), \
    playlist_id varchar(30), \
    danceability decimal(2,3), \
    energy decimal(2,3), \
    key decimal(2), \
    loudness decimal(2,4), \
    mode numeric(1), \
    speechiness decimal(2,5), \
    acousticness decimal(2,5), \
    instrumentalness varchar(10), \
    liveness decimal(2,5), \
    valence decimal(2,4), \
    tempo decimal(3,4), \
    duration_ms numeric(10), \
    PRIMARY KEY (track_id), \
    CONSTRAINT FK_Track_Artist FOREIGN KEY (track_artist) REFERENCES Songs.Artist (artist_name), \
    CONSTRAINT FK_Track_AlbumID FOREIGN KEY (track_album_id) REFERENCES Songs.Album (album_id), \
    CONSTRAINT FK_Track_PlaylistID FOREIGN KEY (playlist_id) REFERENCES Songs.Playlist (playlist_id) \
);"

In [85]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
from psycopg2 import connect

HOST = 'localhost' # since postgres runs on local workspace in Docker
USERNAME = "postgres"
PASSWORD = "postgres"
DATABASE = 'postgres'
CONNECTION_STRING = f"dbname='{DATABASE}' host='{HOST}' user='{USERNAME}' password='{PASSWORD}'"
 
connection = connect(CONNECTION_STRING)

if connection is not None and connection.status == 1:
    print('Connection Successful !!')
else:
    print("Connection Failed (X)")

cursor = connection.cursor() # we will use cursor for all DB operations

Connection Successful !!


In [87]:
with open("raw_schema.sql", encoding="utf-8", mode="r") as schema:
    cursor.execute(schema.read())
    connection.commit()

In [88]:
file_path = "spotify.csv"
lines = []

with open(file_path, encoding="utf-8", mode="r") as file:
    lines = file.readlines()

lines.pop(0) # Removing header

'track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms\n'

In [89]:
class Album:
    album_id: str
    album_name: str
    album_release_date: str

    def __init__(self, id, name=None, release_date=None):
        self.album_id = id
        self.album_name = name
        self.album_release_date = release_date

class Artist:
    artist: str

    def __init__(self, artist):
        self.artist = artist
        

class Genre:
    genre: str

    def __init__(self, genre):
        self.genre = genre
    

class Playlist:
    playlist_id: str
    playlist_name: str
    playlist_genre: Genre
    playlist_subgenre: Genre

    def __init__(self, id, name = None, genre = None, subgenre = None):
        self.playlist_id = id
        self.playlist_name = name
        self.playlist_genre = genre
        self.playlist_subgenre = subgenre
    
class Track:
    track_id: str
    track_name: str
    track_popularity: str
    danceability: str
    energy: str
    key: str
    loudness: str
    mode: str
    speechiness: str
    acousticness: str
    instrumentalness: str
    liveness: str
    valence: str
    tempo: str
    duration_ms: str
    album: Album
    artist: Artist
    playlist: Playlist

    def __init__(self, id, name=None, popularity = None, danceability = None, energy = None, key= None, loudness = None, \
                 mode = None, speechiness = None,  acousticness=None, instrumentalness=None, \
                    liveness=None, valence=None, tempo=None, duration_ms=None,  album=None, artist = None, playlist = None):
        self.track_id = id
        self.track_name = name
        self.track_popularity = popularity
        self.danceability = danceability
        self.energy = energy
        self.key = key
        self.loudness = loudness
        self.mode = mode
        self.speechiness = speechiness
        self.acousticness = acousticness
        self.instrumentalness = instrumentalness
        self.liveness = liveness
        self.valence = valence
        self.tempo = tempo
        self.duration_ms = duration_ms

    def set_album(self, album:Album):
        self.album = album

    
    def set_artist(self, artist:Artist):
        self.artist = artist

    
    def set_playlist(self, playlist:Playlist):
        self.playlist = playlist


In [90]:
def get_album_from_line(line):
    row = line.split(',')
    return Album(row[4], row[5], row[6])


def get_artist_from_line(line):
    row = line.split(',')
    return Artist(row[2])


def get_genre_from_line(line):
    row = line.split(',')
    return Genre(row[9])


def get_subgenre_from_line(line):
    row = line.split(',')
    return Genre(row[10])



def get_playlist_from_line(line):
    row = line.split(',')
    return Playlist(id=row[7], name=row[8], genre=get_genre_from_line(line), subgenre=get_subgenre_from_line(line))


def get_track_from_line(line):
    row = line.split(',')
    track = Track(id=row[0], name=row[1], popularity=row[3], danceability=row[11], energy=row[12], key = row[13], loudness=row[14], \
                 mode=row[15], speechiness=row[16], acousticness=row[17], instrumentalness=row[18], \
                    liveness=row[19], valence=row[20], tempo=row[21], duration_ms=row[22] )
    track.set_album(get_album_from_line(line))
    track.set_artist(get_artist_from_line(line))
    track.set_playlist(get_playlist_from_line(line))
    return track


In [91]:
albums = set()
artists = set()
genres = set()
playlists = set()
tracks = set()
albums_to_save = []
artists_to_save = []
genres_to_save = []
playlists_to_save = []
tracks_to_save = []


for line in lines:
    album = get_album_from_line(line)

    if album.album_id not in albums:
        albums.add(album.album_id)
        albums_to_save.append(album)

    artist = get_artist_from_line(line)

    if artist.artist not in artists:
        artists.add(artist.artist)
        artists_to_save.append(artist)

    genre = get_genre_from_line(line)

    if genre.genre not in genres:
        genres.add(genre.genre)
        genres_to_save.append(genre)

    subgenre = get_subgenre_from_line(line)

    if subgenre.genre not in genres:
        genres.add(subgenre.genre)
        genres_to_save.append(subgenre)

    playlist = get_playlist_from_line(line)

    if playlist.playlist_id not in playlists:
        playlists.add(playlist.playlist_id)
        playlists_to_save.append(playlist)

    track = get_track_from_line(line)

    if track.track_id not in tracks:
        tracks.add(track.track_id)
        tracks_to_save.append(track)

In [92]:
INSERT_ARTIST = """INSERT INTO Songs.Artist (artist_name) VALUES (%s)"""
INSERT_ALBUM = """INSERT INTO Songs.Album (album_id, album_name, album_release_date) VALUES (%s, %s, %s)"""
INSERT_GENRE = """INSERT INTO Songs.Genre (genre_name) VALUES (%s)"""
INSERT_PLAYLIST = """INSERT INTO Songs.Playlist (playlist_id, playlist_name, playlist_genre, playlist_subgenre) VALUES (%s,%s,%s,%s)"""
INSERT_TRACK = """INSERT INTO Songs.Track (track_id, track_name, track_artist, track_popularity, track_album_id, playlist_id, danceability, \
energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms) \
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) """


In [93]:
for i in artists_to_save:
    cursor.execute(INSERT_ARTIST, [i.artist])

for i in albums_to_save:
    cursor.execute(INSERT_ALBUM, [i.album_id, i.album_name, i.album_release_date])

for i in genres_to_save:
    cursor.execute(INSERT_GENRE, [i.genre])

for i in playlists_to_save:
    cursor.execute(INSERT_PLAYLIST, [i.playlist_id, i.playlist_name, i.playlist_genre.genre, i.playlist_subgenre.genre])

for i in tracks_to_save:
    cursor.execute(INSERT_TRACK, [i.track_id, i.track_name, i.artist.artist, i.track_popularity, i.album.album_id, i.playlist.playlist_id, i.danceability, \
i.energy, i.key, i.loudness, i.mode, i.speechiness, i.acousticness, i.instrumentalness, i.liveness, i.valence, i.tempo, i.duration_ms])


In [94]:
cursor.execute("SELECT * FROM SONGS.Track")

for i in cursor.fetchall():
    print(i)

('6f807x0ima9a1j3VPbc7VN', "I Don't Care (with Justin Bieber) - Loud Luxury Remix", 'Ed Sheeran', Decimal('66'), '2oCs0DGTsRO98Gh5ZSl2Cx', 'Pop Remix', Decimal('0.748'), Decimal('0.916'), Decimal('6'), Decimal('-2.634'), Decimal('1'), Decimal('0.0583'), Decimal('0.102'), '0', Decimal('0.0653'), Decimal('0.518'), Decimal('122.036'), Decimal('194754'))
('0r7CVbZTWZgbTCYdfa2P31', 'Memories - Dillon Francis Remix', 'Maroon 5', Decimal('67'), '63rPSO264uRjW1X5E6cWv6', 'Pop Remix', Decimal('0.726'), Decimal('0.815'), Decimal('11'), Decimal('-4.969'), Decimal('1'), Decimal('0.0373'), Decimal('0.0724'), '0.00421', Decimal('0.357'), Decimal('0.693'), Decimal('99.972'), Decimal('162600'))
('1z1Hg7Vb0AhHDiEmnDE79l', 'All the Time - Don Diablo Remix', 'Zara Larsson', Decimal('70'), '1HoSmj2eLcsrR0vE9gThr4', 'Pop Remix', Decimal('0.675'), Decimal('0.931'), Decimal('1'), Decimal('-3.432'), Decimal('0'), Decimal('0.0742'), Decimal('0.0794'), '2.33e-5', Decimal('0.11'), Decimal('0.613'), Decimal('124.

# Refined

In [ ]:
with open("refined_schema.sql", encoding="utf-8", mode="r") as schema:
    cursor.execute(schema.read())
    connection.commit()